In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import pytorch_lightning as pl
from PIL import Image

from pytorch_lightning.loggers import TensorBoardLogger

In [2]:
import config
from dataset import DataModule
from model import ClassificationModel

In [3]:
torch.random.manual_seed(config.RANDOM_SEED)
pl.seed_everything(config.RANDOM_SEED)

Seed set to 42


42

In [4]:
dm = DataModule(
    data_path=config.DATA_PATH,
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    train_test_ratio=config.TRAIN_TEST_RATIO,
    train_val_ratio=config.TRAIN_VAL_RATIO,
)

model = ClassificationModel(learning_rate=config.LEARNING_RATE)

logger = TensorBoardLogger("tb_logs", name="my_model")

trainer = pl.Trainer(
    accelerator=config.ACCELERATOR,
    devices=config.DEVICES,
    min_epochs=config.MIN_EPOCHS,
    max_epochs=config.MAX_EPOCHS,
    enable_checkpointing=False,
    logger=logger,
)

/Users/evlko/Documents/GitHub/Game-Interface-Flows-ML/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/evlko/Documents/GitHub/Game-Interface-Flows-ML/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
trainer.fit(model, dm)
trainer.validate(model, dm)
trainer.test(model, dm)


  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | loss_fn   | CrossEntropyLoss | 0     
2 | accuracy  | BinaryAccuracy   | 0     
3 | precision | BinaryPrecision  | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Epoch 4: 100%|██████████| 50/50 [00:28<00:00,  1.72it/s, v_num=9, val_acc=0.909, val_precision=0.850]

`Trainer.fit` stopped: `max_epochs=5` reached.


Validation DataLoader 0: 100%|██████████| 10/10 [00:01<00:00,  5.51it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc             0.903124988079071
        val_loss            0.2872771620750427
      val_precision         0.8376444578170776
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00,  6.87it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc         

[{'test_loss': 0.28641724586486816,
  'test_acc': 0.8999999761581421,
  'test_precision': 0.8333333134651184}]

In [18]:
test_loader = dm.train_dataloader()
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on test images: {100 * correct / total} %")
print(correct)

Accuracy of the network on test images: 90.75 %
1452


In [ ]:
# сумма вышла (71 + 289 + ...) / 2000 = 90%

In [13]:
def count_images_in_dataloader(dataloader):
    total_images = 0
    for _, labels in dataloader:
        total_images += labels.size(
            0
        )  # Assuming labels are batched similarly to images
    return total_images


# Example usage with your test DataLoader
test_loader = dm.train_dataloader()
total_images = count_images_in_dataloader(test_loader)
print(f"Total number of images in the test DataLoader: {total_images}")

Total number of images in the test DataLoader: 1600


In [14]:
80 + 320 + 1600  # something is wrong...

2000

In [22]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import albumentations as A
import pytorch_lightning as pl
from albumentation_transforms import AlbumentationTransforms
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, random_split
from torchvision import datasets

# Step 1: Setup the transformations, ensure these are the same as used in your training minus any augmentations
transform = AlbumentationTransforms(
    A.Compose(
        [
            A.Resize(width=256, height=256),
            A.ToGray(always_apply=True),
            A.Normalize(mean=[0.485], std=[0.229]),
            ToTensorV2(),
        ]
    )
)

# Step 2: Load your entire dataset
all_data = datasets.ImageFolder(root="data/", transform=transform)

# Step 3: Create a DataLoader
all_data_loader = DataLoader(all_data, batch_size=32, shuffle=False, num_workers=4)

# Step 4: Calculate the accuracy
correct = 0
total = 0
model.eval()  # Ensure the model is in evaluation mode

with torch.no_grad():
    for images, labels in all_data_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy of the network on all images: {accuracy:.2f} %")

Accuracy of the network on all images: 90.30 %


In [26]:
import numpy as np


def get_transform():
    return A.Compose(
        [
            A.Resize(width=256, height=256),
            A.ToGray(always_apply=True),
            A.Normalize(mean=[0.485], std=[0.229]),
            ToTensorV2(),
        ]
    )


def load_and_transform_image(image_path, transform):
    image = Image.open(image_path)
    image = image.convert("RGB")
    image_np = np.array(image)
    transformed = transform(image=image_np)["image"]
    transformed = transformed.unsqueeze(0)
    return transformed


# Define the path to your image
image_path = "data/1/anthem_1.jpg"

# Get the transformation function ready
transform = get_transform()

# Load and transform the image
transformed_image = load_and_transform_image(image_path, transform)

# Predict with the model
model.eval()
with torch.no_grad():
    outputs = model(transformed_image)
    _, predicted = torch.max(outputs.data, 1)

# Print or return the result
print(f"Predicted class: {predicted.item()}")

Predicted class: 1


In [34]:
torch.save(model.state_dict(), "resnet18_weights.pth")

import torch
import torch.nn as nn
from torchvision import models


def create_model(num_classes=2):
    # Load a pretrained ResNet18 model
    model = models.resnet18(weights=None)
    # Modify the fully connected layer to match the number of classes
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model


saved_model = create_model(num_classes=2)

state_dict = torch.load("resnet18_weights.pth")

adjusted_state_dict = {
    key.replace("model.", ""): value for key, value in state_dict.items()
}

saved_model.load_state_dict(adjusted_state_dict)

saved_model.eval()


def predict_image(image_path, model, transform):
    image = load_and_transform_image(image_path, transform)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()


image_path = "data/1/anthem_1.jpg"
predicted_class = predict_image(image_path, model, get_transform())
print(f"Predicted class: {predicted_class}")

Predicted class: 1
